In [ ]:
# Final with GUI working

import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import tkinter as tk
from PIL import Image, ImageTk

class Application:
    def __init__(self):
        self.vs = cv2.VideoCapture(0)
        self.root = tk.Tk()
        self.root.title("Sign language to Text Converter")
        self.root.geometry("1200x1100")

        self.panel = tk.Label(self.root)
        self.panel.place(x=135, y=60, width=470, height=470)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=690, y=60, width=310, height=310)

        self.T = tk.Label(self.root)
        self.T.place(x=31, y=17)
        self.T.config(text="Sign Language to Text", font=("Courier", 25, "bold"))

        self.panel3 = tk.Label(self.root)
        self.panel3.place(x=500, y=640)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=640)
        self.T1.config(text="Character:", font=("Courier", 40, "bold"))

        self.character_label = tk.Label(self.root)
        self.character_label.place(x=350, y=640)
        self.character_label.config(font=("Courier", 40, "bold"))

        self.panel4 = tk.Label(self.root)
        self.panel4.place(x=220, y=700)

        self.T2 = tk.Label(self.root)
        self.T2.place(x=10, y=700)
        self.T2.config(text="Word:", font=("Courier", 40, "bold"))

        self.word_label = tk.Label(self.root)
        self.word_label.place(x=350, y=700)
        self.word_label.config(font=("Courier", 40, "bold"))

        self.panel5 = tk.Label(self.root)
        self.panel5.place(x=350, y=760)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=760)
        self.T3.config(text="Sentence:", font=("Courier", 40, "bold"))

        self.current_character = ""
        self.current_word = ""

        self.video_loop()

    def video_loop(self):
        detector = HandDetector(maxHands=1)
        classifier = Classifier("C:/Users/tanma/OneDrive/Desktop/Sign Language youtube/converted_keras/keras_model.h5",
                                "C:/Users/tanma/OneDrive/Desktop/Sign Language youtube/converted_keras/labels.txt")
        offset = 20
        imgSize = 300

        labels = ["0", "1", "2"]

        while True:
            success, img = self.vs.read()
            imgOutput = img.copy()
            hands, img = detector.findHands(img)
            if hands:
                hand = hands[0]
                x, y, w, h = hand['bbox']

                imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
                imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

                if not imgCrop.any():
                    continue

                imgCropShape = imgCrop.shape

                aspectRatio = h / w

                if aspectRatio > 1:
                    k = imgSize / h
                    wCal = math.ceil(k * w)
                    imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                    imgResizeShape = imgResize.shape
                    wGap = math.ceil((imgSize - wCal) / 2)
                    imgWhite[:, wGap:wCal + wGap] = imgResize
                    prediction, index = classifier.getPrediction(imgWhite, draw=False)
                    print(index)

                else:
                    k = imgSize / w
                    hCal = math.ceil(k * h)
                    imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                    imgResizeShape = imgResize.shape
                    hGap = math.ceil((imgSize - hCal) / 2)
                    imgWhite[hGap:hCal + hGap, :] = imgResize
                    prediction, index = classifier.getPrediction(imgWhite, draw=False)

                confidence_threshold = 0.9  # Adjust based on your observations
                if np.max(prediction) < confidence_threshold:
                    recognized_text = "Not Recognized"
                else:
                    recognized_text = labels[index]

                    # Update current character and word
                    if recognized_text != self.current_character:
                        self.current_character = recognized_text
                        self.current_word += recognized_text

                print(recognized_text)

                cv2.rectangle(imgOutput, (x - offset, y - offset - 50),
                              (x - offset + 90, y - offset - 50 + 50), (255, 0, 255), cv2.FILLED)
                cv2.putText(imgOutput, recognized_text, (x, y - 26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
                cv2.rectangle(imgOutput, (x - offset, y - offset),
                              (x + w + offset, y + h + offset), (255, 0, 255), 4)

                # Convert the OpenCV image to RGB format for use with Tkinter
                imgRGB = cv2.cvtColor(imgOutput, cv2.COLOR_BGR2RGB)
                # Convert the NumPy array to an ImageTk PhotoImage
                imgTk = ImageTk.PhotoImage(Image.fromarray(imgRGB))

                # Update the image in the Tkinter label
                self.panel.img = imgTk
                self.panel.config(image=imgTk)

                # Convert the imgWhite frame to RGB format for use with Tkinter
                imgWhiteRGB = cv2.cvtColor(imgWhite, cv2.COLOR_BGR2RGB)
                # Convert the NumPy array to an ImageTk PhotoImage
                imgWhiteTk = ImageTk.PhotoImage(Image.fromarray(imgWhiteRGB))

                # Update the imgWhite image in the Tkinter label
                self.panel2.img = imgWhiteTk
                self.panel2.config(image=imgWhiteTk)

                # Update the recognized text in the Tkinter label
                self.character_label.config(text=self.current_character)

                # Update the word in the Tkinter label
                self.word_label.config(text=self.current_word)

            # Update the Tkinter window
            self.root.update()

    def run(self):
        self.root.mainloop()

if __name__ == '__main__':
    app = Application()
    app.run()





1/1 [==============================] - 2s 2s/step
0
Not Recognized
1/1 [==============================] - 0s 48ms/step
0
Not Recognized
1/1 [==============================] - 0s 45ms/step
0
1/1 [==============================] - 0s 46ms/step
0
0
1/1 [==============================] - 0s 48ms/step
0
0
1/1 [==============================] - 0s 70ms/step
0
0
1/1 [==============================] - 0s 40ms/step
0
0
1/1 [==============================] - 0s 86ms/step
0
0
1/1 [==============================] - 0s 52ms/step
0
0
1/1 [==============================] - 0s 62ms/step
0
0
1/1 [==============================] - 0s 42ms/step
0
0
1/1 [==============================] - 0s 42ms/step
0
0
1/1 [==============================] - 0s 48ms/step
0
0
1/1 [==============================] - 0s 46ms/step
1
Not Recognized
1/1 [==============================] - 0s 53ms/step
1
Not Recognized
1/1 [==============================] - 0s 48ms/step
1
Not Recognized
1/1 [==============================] - 0

1/1 [==============================] - 0s 40ms/step
0
Not Recognized
1/1 [==============================] - 0s 47ms/step
0
Not Recognized
1/1 [==============================] - 0s 35ms/step
0
Not Recognized
1/1 [==============================] - 0s 54ms/step
0
Not Recognized
1/1 [==============================] - 0s 42ms/step
0
0
1/1 [==============================] - 0s 47ms/step
0
Not Recognized


In [ ]:
## OG

import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import tkinter as tk

class application:
    def __init__(self):
        self.vs = cv2.VideoCapture(0)
        self.root = tk.Tk()
        self.root.title("Sign language to Text Converter")
        self.root.geometry("900x1100")
        self.panel = tk.Label(self.root)
        self.panel.place(x=135, y=10, width=640, height=640)
        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=460, y=95, width=310, height=310)
        
        self.T = tk.Label(self.root)
        self.T.place(x=31, y=17)
        self.T.config(text="Sign Language to Text", font=("Courier", 40, "bold"))
        self.panel3 = tk.Label(self.root)
        self.panel3.place(x=500, y=640)
        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=640)
        self.T1.config(text="Character:", font=("Courier", 40, "bold"))
        self.panel4 = tk.Label(self.root)
        self.panel4.place(x=220, y=700)
        self.T2 = tk.Label(self.root)
        self.T2.place(x=10, y=700)
        self.T2.config(text="Word:", font=("Courier", 40, "bold"))
        self.panel5 = tk.Label(self.root)
        self.panel5.place(x=350, y=760)
        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=760)
        self.T3.config(text="Sentence:", font=("Courier", 40, "bold"))
        self.video_loop()
        
        
    def video_loop(self):
        detector = HandDetector(maxHands=1)
        classifier = Classifier("C:/Users/tanma/OneDrive/Desktop/Sign Language youtube/converted_keras/keras_model.h5", "C:/Users/tanma/OneDrive/Desktop/Sign Language youtube/converted_keras/labels.txt")
        offset = 20
        imgSize = 300

        folder = "Data/C"
        counter = 0

        labels = ["0", "1", "2"]

        while True:
            success, img = self.vs.read()
            imgOutput = img.copy()
            hands, img = detector.findHands(img)
            if hands:
                hand = hands[0]
                x, y, w, h = hand['bbox']

                imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
                imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

                if not imgCrop.any():
                    continue

                imgCropShape = imgCrop.shape

                aspectRatio = h / w

                if aspectRatio > 1:
                    k = imgSize / h
                    wCal = math.ceil(k * w)
                    imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                    imgResizeShape = imgResize.shape
                    wGap = math.ceil((imgSize - wCal) / 2)
                    imgWhite[:, wGap:wCal + wGap] = imgResize
                    prediction, index = classifier.getPrediction(imgWhite, draw=False)
                    print(index)

                else:
                    k = imgSize / w
                    hCal = math.ceil(k * h)
                    imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                    imgResizeShape = imgResize.shape
                    hGap = math.ceil((imgSize - hCal) / 2)
                    imgWhite[hGap:hCal + hGap, :] = imgResize
                    prediction, index = classifier.getPrediction(imgWhite, draw=False)

                cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                              (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
                cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
                cv2.rectangle(imgOutput, (x-offset, y-offset),
                              (x + w+offset, y + h+offset), (255, 0, 255), 4)

                cv2.imshow("ImageCrop", imgCrop)
                cv2.imshow("ImageWhite", imgWhite)

            cv2.imshow("Image", imgOutput)
            cv2.waitKey(1)
            pass
    def run(self):
        self.video_loop()
        self.root.mainloop()
        
if __name__ == '__main__':
    app = application()
    app.run()


Exception ignored in: <function PhotoImage.__del__ at 0x000001E757867D80>
Traceback (most recent call last):
  File "C:\Users\tanma\anaconda3\Lib\site-packages\PIL\ImageTk.py", line 133, in __del__
    name = self.__photo.name
           ^^^^^^^^^^^^
AttributeError: 'PhotoImage' object has no attribute '_PhotoImage__photo'
Exception ignored in: <function PhotoImage.__del__ at 0x000001E757867D80>
Traceback (most recent call last):
  File "C:\Users\tanma\anaconda3\Lib\site-packages\PIL\ImageTk.py", line 133, in __del__
    name = self.__photo.name
           ^^^^^^^^^^^^
AttributeError: 'PhotoImage' object has no attribute '_PhotoImage__photo'


1/1 [==============================] - 1s 867ms/step
0
1/1 [==============================] - 0s 35ms/step
1
1/1 [==============================] - 0s 36ms/step
1
1/1 [==============================] - 0s 40ms/step
1
1/1 [==============================] - 0s 39ms/step
1
1/1 [==============================] - 0s 40ms/step
1
1/1 [==============================] - 0s 39ms/step
1
1/1 [==============================] - 0s 35ms/step
1
1/1 [==============================] - 0s 36ms/step
1
1/1 [==============================] - 0s 37ms/step
1
1/1 [==============================] - 0s 32ms/step
1
1/1 [==============================] - 0s 32ms/step
1
1/1 [==============================] - 0s 36ms/step
1
1/1 [==============================] - 0s 38ms/step
1
1/1 [==============================] - 0s 38ms/step
1
1/1 [==============================] - 0s 40ms/step
1
1/1 [==============================] - 0s 40ms/step
1
1/1 [==============================] - 0s 41ms/step
1
1/1 [======================

1/1 [==============================] - 0s 35ms/step
0
1/1 [==============================] - 0s 41ms/step


In [ ]:
## OG with iin GUI
### Almost Final wala


import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import tkinter as tk
from PIL import Image, ImageTk

class Application:
    def __init__(self):
        self.vs = cv2.VideoCapture(0)
        self.root = tk.Tk()
        self.root.title("Sign language to Text Converter")
        self.root.geometry("1200x1100")

        self.panel = tk.Label(self.root)
        self.panel.place(x=135, y=60, width=470, height=470)

        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=690, y=60, width=310, height=310)

        self.T = tk.Label(self.root)
        self.T.place(x=31, y=17)
        self.T.config(text="Sign Language to Text", font=("Courier", 25, "bold"))

        self.panel3 = tk.Label(self.root)
        self.panel3.place(x=500, y=640)

        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=640)
        self.T1.config(text="Character:", font=("Courier", 40, "bold"))

        self.panel4 = tk.Label(self.root)
        self.panel4.place(x=220, y=700)

        self.T2 = tk.Label(self.root)
        self.T2.place(x=10, y=700)
        self.T2.config(text="Word:", font=("Courier", 40, "bold"))

        self.panel5 = tk.Label(self.root)
        self.panel5.place(x=350, y=760)

        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=760)
        self.T3.config(text="Sentence:", font=("Courier", 40, "bold"))

        self.video_loop()

    def video_loop(self):
        detector = HandDetector(maxHands=1)
        classifier = Classifier("C:/Users/tanma/OneDrive/Desktop/Sign Language youtube/converted_keras/keras_model.h5",
                                "C:/Users/tanma/OneDrive/Desktop/Sign Language youtube/converted_keras/labels.txt")
        offset = 20
        imgSize = 300

        folder = "Data/C"
        counter = 0

        labels = ["0", "1", "2"]

        while True:
            success, img = self.vs.read()
            imgOutput = img.copy()
            hands, img = detector.findHands(img)
            if hands:
                hand = hands[0]
                x, y, w, h = hand['bbox']

                imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
                imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

                if not imgCrop.any():
                    continue

                imgCropShape = imgCrop.shape

                aspectRatio = h / w

                if aspectRatio > 1:
                    k = imgSize / h
                    wCal = math.ceil(k * w)
                    imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                    imgResizeShape = imgResize.shape
                    wGap = math.ceil((imgSize - wCal) / 2)
                    imgWhite[:, wGap:wCal + wGap] = imgResize
                    prediction, index = classifier.getPrediction(imgWhite, draw=False)
                    print(index)

                else:
                    k = imgSize / w
                    hCal = math.ceil(k * h)
                    imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                    imgResizeShape = imgResize.shape
                    hGap = math.ceil((imgSize - hCal) / 2)
                    imgWhite[hGap:hCal + hGap, :] = imgResize
                    prediction, index = classifier.getPrediction(imgWhite, draw=False)
                    
                confidence_threshold = 0.9  # Adjust based on your observations
                if np.max(prediction) < confidence_threshold:
                    label_text = "Not Recognized"
                else:
                    label_text = labels[index]
                    
                print(label_text)

                cv2.rectangle(imgOutput, (x - offset, y - offset - 50),
                              (x - offset + 90, y - offset - 50 + 50), (255, 0, 255), cv2.FILLED)
                cv2.putText(imgOutput, label_text, (x, y - 26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
                cv2.rectangle(imgOutput, (x - offset, y - offset),
                              (x + w + offset, y + h + offset), (255, 0, 255), 4)

                # Convert the OpenCV image to RGB format for use with Tkinter
                imgRGB = cv2.cvtColor(imgOutput, cv2.COLOR_BGR2RGB)
                # Convert the NumPy array to an ImageTk PhotoImage
                imgTk = ImageTk.PhotoImage(Image.fromarray(imgRGB))

                # Update the image in the Tkinter label
                self.panel.img = imgTk
                self.panel.config(image=imgTk)

                # Convert the imgWhite frame to RGB format for use with Tkinter
                imgWhiteRGB = cv2.cvtColor(imgWhite, cv2.COLOR_BGR2RGB)
                # Convert the NumPy array to an ImageTk PhotoImage
                imgWhiteTk = ImageTk.PhotoImage(Image.fromarray(imgWhiteRGB))

                # Update the imgWhite image in the Tkinter label
                self.panel2.img = imgWhiteTk
                self.panel2.config(image=imgWhiteTk)

            # Update the Tkinter window
            self.root.update()

    def run(self):
        self.root.mainloop()

if __name__ == '__main__':
    app = Application()
    app.run()





1/1 [==============================] - 1s 1s/step
2
2
1/1 [==============================] - 0s 43ms/step
2
2
1/1 [==============================] - 0s 50ms/step
2
2
1/1 [==============================] - 0s 86ms/step
2
2
1/1 [==============================] - 0s 47ms/step
2
2
1/1 [==============================] - 0s 51ms/step
2
2
1/1 [==============================] - 0s 45ms/step
2
2
1/1 [==============================] - 0s 46ms/step
2
2
1/1 [==============================] - 0s 50ms/step
2
2
1/1 [==============================] - 0s 52ms/step
2
2
1/1 [==============================] - 0s 67ms/step
2
2
1/1 [==============================] - 0s 55ms/step
2
2
1/1 [==============================] - 0s 42ms/step
2
2
1/1 [==============================] - 0s 45ms/step
2
2
1/1 [==============================] - 0s 46ms/step
2
2
1/1 [==============================] - 0s 44ms/step
2
2
1/1 [==============================] - 0s 46ms/step
2
2
1/1 [==============================] - 0s 95ms/

1/1 [==============================] - 0s 44ms/step
2
2
1/1 [==============================] - 0s 67ms/step
2
2
1/1 [==============================] - 0s 40ms/step
2
2
1/1 [==============================] - 0s 48ms/step
2
2
1/1 [==============================] - 0s 48ms/step
2
2
1/1 [==============================] - 0s 46ms/step
2
2
1/1 [==============================] - 0s 77ms/step
2
2
1/1 [==============================] - 0s 55ms/step
2
2
1/1 [==============================] - 0s 41ms/step
2
2
1/1 [==============================] - 0s 43ms/step
2
2
1/1 [==============================] - 0s 48ms/step
2
2
1/1 [==============================] - 0s 39ms/step
2
2
1/1 [==============================] - 0s 46ms/step
2
2
1/1 [==============================] - 0s 55ms/step
2
2
1/1 [==============================] - 0s 41ms/step
2
2
1/1 [==============================] - 0s 47ms/step
2
2
1/1 [==============================] - 0s 46ms/step
2
2
1/1 [==============================] - 0s 65ms/s

In [2]:
## smooth

import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math
import tkinter as tk
import time

class application:
    def __init__(self):
        self.vs = cv2.VideoCapture(0)
        self.vs.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        self.vs.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

        self.root = tk.Tk()
        self.root.title("Sign language to Text Converter")
        self.root.geometry("900x1100")
        self.panel = tk.Label(self.root)
        self.panel.place(x=135, y=10, width=640, height=480)
        self.panel2 = tk.Label(self.root)
        self.panel2.place(x=460, y=95, width=310, height=310)

        self.T = tk.Label(self.root)
        self.T.place(x=31, y=17)
        self.T.config(text="Sign Language to Text", font=("Courier", 40, "bold"))
        self.panel3 = tk.Label(self.root)
        self.panel3.place(x=500, y=640)
        self.T1 = tk.Label(self.root)
        self.T1.place(x=10, y=640)
        self.T1.config(text="Character:", font=("Courier", 40, "bold"))
        self.panel4 = tk.Label(self.root)
        self.panel4.place(x=220, y=700)
        self.T2 = tk.Label(self.root)
        self.T2.place(x=10, y=700)
        self.T2.config(text="Word:", font=("Courier", 40, "bold"))
        self.panel5 = tk.Label(self.root)
        self.panel5.place(x=350, y=760)
        self.T3 = tk.Label(self.root)
        self.T3.place(x=10, y=760)
        self.T3.config(text="Sentence:", font=("Courier", 40, "bold"))

        self.detector = HandDetector(maxHands=1)
        self.classifier = Classifier("C:/Users/tanma/OneDrive/Desktop/Sign Language youtube/converted_keras/keras_model.h5", "C:/Users/tanma/OneDrive/Desktop/Sign Language youtube/converted_keras/labels.txt")

        self.offset = 20
        self.imgSize = 300
        self.labels = ["0", "1", "2"]
        self.delay_duration = 10  # seconds

    def process_frame(self, frame):
        imgOutput = frame.copy()
        hands, _ = self.detector.findHands(frame)

        if hands:
            hand = hands[0]
            x, y, w, h = hand['bbox']

            imgWhite = np.ones((self.imgSize, self.imgSize, 3), np.uint8) * 255
            imgCrop = frame[y - self.offset:y + h + self.offset, x - self.offset:x + w + self.offset]

            if not imgCrop.any():
                return imgOutput

            aspectRatio = h / w

            if aspectRatio > 1:
                k = self.imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, self.imgSize))
                wGap = math.ceil((self.imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
                prediction, index = self.classifier.getPrediction(imgWhite, draw=False)
                print(index)
                print("Confidence values:", prediction)
            else:
                k = self.imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (self.imgSize, hCal))
                hGap = math.ceil((self.imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
                prediction, index = self.classifier.getPrediction(imgWhite, draw=False)

            # Check confidence threshold
            confidence_threshold = 0.8  # Adjust based on your observations
            if np.max(prediction) < confidence_threshold:
                label_text = "Not Recognized"
            else:
                label_text = self.labels[index]

                # Introduce a delay after each successful detection
                time.sleep(self.delay_duration)
            print(label_text)

            cv2.rectangle(imgOutput, (x - self.offset, y - self.offset-50),
                          (x - self.offset+90, y - self.offset-50+50), (255, 0, 255), cv2.FILLED)
            cv2.putText(imgOutput, label_text, (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
            cv2.rectangle(imgOutput, (x-self.offset, y-self.offset),
                          (x + w+self.offset, y + h+self.offset), (255, 0, 255), 4)

            cv2.imshow("ImageCrop", imgCrop)
            cv2.imshow("ImageWhite", imgWhite)

        return imgOutput

    def video_loop(self):
        while True:
            ret, frame = self.vs.read()

            if not ret:
                break

            frame = cv2.flip(frame, 1)  # Flip the frame horizontally for a more natural hand movement
            processed_frame = self.process_frame(frame)

            cv2.imshow("Image", processed_frame)
            if cv2.waitKey(1) == ord('q'):
                break

    def run(self):
        self.video_loop()
        self.vs.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    app = application()
    app.run()





1/1 [==============================] - 1s 1s/step
1
Confidence values: [0.013300719, 0.9862242, 0.00047504096]
1
1/1 [==============================] - 0s 58ms/step
1
Confidence values: [0.02407999, 0.88173145, 0.09418851]
1
1/1 [==============================] - 0s 42ms/step
1
Confidence values: [0.020333495, 0.61021495, 0.36945158]
Not Recognized
1/1 [==============================] - 0s 56ms/step
2
Confidence values: [0.003507258, 0.38452643, 0.6119663]
Not Recognized
1/1 [==============================] - 0s 47ms/step
1
Confidence values: [0.0024016397, 0.52874994, 0.46884838]
Not Recognized
1/1 [==============================] - 0s 51ms/step
1
Confidence values: [0.003698692, 0.5870253, 0.409276]
Not Recognized
1/1 [==============================] - 0s 39ms/step
1
Confidence values: [0.0018602221, 0.68097496, 0.3171648]
Not Recognized
1/1 [==============================] - 0s 53ms/step
2
Confidence values: [0.0024994733, 0.33324537, 0.66425514]
Not Recognized
1/1 [=============